In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import json as json
import matplotlib.pyplot as plt

# File to Load (Remember to Change These)
crime_2018 = "Resources/2018-19-data_sa_crime.csv"
crime_2019 = "Resources/2019-20-fullyr-data_sa_crime (1).csv"
crime_2020 = "Resources/2020-21_crime_qtr1_and_qtr2 (1).csv"
sa_postcodes = "Resources/australian_postcodes (1).csv"

# Read data files and store into DataFrames
crime_2018_df = pd.read_csv(crime_2018, parse_dates = ['Reported Date'])
crime_2019_df = pd.read_csv(crime_2019, parse_dates = ['Reported Date'])
crime_2020_df = pd.read_csv(crime_2020, parse_dates = ['Reported Date'])
sa_postcodes_df = pd.read_csv(sa_postcodes)

In [ ]:
#merging 4 crime datasets into a merged_data dataframe
merged_data = pd.concat([crime_2018_df,crime_2019_df,crime_2020_df], ignore_index=True)

In [ ]:
#cleaning merged_data dataframe and removing rows containing null values
clean_crime_data = merged_data.dropna(how='any')

In [ ]:
#renaming SA postcode set to allow merge on shared column
sa_postcodes_df = sa_postcodes_df.rename(columns = {"postcode":"Postcode - Incident",
                                                   'locality': "Suburb - Incident"})

In [ ]:
#using loc function to remove any crimes listed against postcode "NOT DISCLOSED"
clean_crime_data = clean_crime_data.loc[clean_crime_data["Postcode - Incident"]!= "NOT DISCLOSED"].copy()

In [ ]:
#changing datatype from object to integer to allow merge of two datasets
clean_crime_data["Postcode - Incident"] = clean_crime_data["Postcode - Incident"].astype(str).astype(int)

In [ ]:
#Check datatypes of clean_crime_data
#clean_crime_data.dtypes

In [ ]:
#Check datatypes of sa_postcodes_df
#sa_postcodes_df.dtypes

In [ ]:
#merge both datasets - using left join to drop any postcode data not associated with crimes in SA
crime_postcode_merge_df = pd.merge(clean_crime_data, sa_postcodes_df, on = ["Postcode - Incident", "Suburb - Incident"], how = "left")

In [ ]:
#review updated dataframe
crime_postcode_merge_df.head()

In [ ]:
crime_postcode_merge_df = crime_postcode_merge_df.drop(columns=['SA2_MAINCODE_2016', 'type', 'sa3', 'SA3_CODE_2016', 
                                      'SA1_MAINCODE_2011', 'id', 'SA1_MAINCODE_2016',
                                      'SA4_CODE_2016','RA_2011','RA_2016','MMM_2015','MMM_2019',
                                      'status','sa4','state'])

In [ ]:
crime_postcode_merge_df.head()

In [ ]:
date_breakdown_df = crime_postcode_merge_df.drop(columns=['SA4_NAME_2016', 'SA3_NAME_2016', 'SA2_NAME_2016', 'Long_precise',
                                                         'Lat_precise', 'region', 'sa4name', 'sa3name', 'dc', 'lat', 'long',
                                                         'Suburb - Incident', 'Postcode - Incident', 'Offence Level 1 Description',
                                                         'Offence Level 2 Description', 'Offence Level 3 Description'])

date_breakdown_df.head()

In [ ]:
date_breakdown_df['Reported Date'] = pd.to_datetime(date_breakdown_df['Reported Date']) 

crime_data_2019_start = date_breakdown_df.loc[date_breakdown_df['Reported Date'] >= '2019/01/01']

In [ ]:
crime_data_2019_all = crime_data_2019_start.loc[date_breakdown_df['Reported Date'] <= '2019/12/31']

In [ ]:
#crime_data_2019_all.head()

crime_data_2019_all['Month'] = crime_data_2019_all['Reported Date'].dt.month

crime_data_2019_all

In [ ]:
#making 2020 DF

crime_data_2020_start = date_breakdown_df.loc[date_breakdown_df['Reported Date'] >= '2020/01/01']

In [ ]:
crime_data_2020_all = crime_data_2020_start.loc[date_breakdown_df['Reported Date'] <= '2020/12/31']


crime_data_2020_all['Month'] = crime_data_2020_all['Reported Date'].dt.month

crime_data_2020_all

In [ ]:
# I now have the data in 2019 (crime_data_2019_all) in one DF and 2020 (crime_data_2020_all) in another 

In [ ]:
grouped_2019_crime = crime_data_2019_all.groupby(['Month'])['Offence count'].count().reset_index()
grouped_2020_crime = crime_data_2020_all.groupby(['Month'])['Offence count'].count().reset_index()

In [ ]:
grouped_2019_crime

In [ ]:
ax = grouped_2019_crime.plot(kind = 'line', x = 'Month', y = 'Offence count', label = '2019 Crime', figsize = (10, 5))
grouped_2020_crime.plot(kind = 'line', x = 'Month', y = 'Offence count', color ='red', xlabel = "Month of the Year", ylabel = 'Offence Count', label = '2020 Crime', ax= ax)
ax.legend()
ax.set_xticks(np.arange(1, 12+1, 1))
plt.grid()
plt.show()